In [3]:
import pandas as pd
from langdetect import detect
from SPARQLWrapper import SPARQLWrapper, JSON
import re
import numpy as np
import sys
import os
import time
from nltk.tag import pos_tag
import urllib.parse
import nltk
# nltk.download('averaged_perceptron_tagger')
from googletrans import Translator
translator = Translator()

In [4]:
target = pd.read_csv("/home/ngiersz/Projects/zti/data/CEA_Round1_Targets.csv",
                    sep = ',',
                     skiprows=range(112,113), # 112 wadliwy wiersz
                    names=['TableID', 'ColumnID', 'RowID'])
target[:3]

,TableID,ColumnID,RowID
0,50245608_0_871275842592178099,0,154
1,39107734_2_2329160387535788734,1,32
2,22864497_0_8632623712684511496,0,227


In [5]:
target.shape

(8417, 3)

In [24]:
start = time.time()

# słownik, z którego będzie tworzony wynikowy df
answers_dict = {"TableID":[],
               "ColumnID":[],
               "RowID":[],
               "URI":[]}

# zmienna, aby ograniczyć liczbę przetwarzanych wierszy, niżej masz 3 linijki skomentowane,
# odkomentowujesz je i ustawiasz sobię liczbę wierszy w tym warunku, aktualnie 1000
counter = 0
# zlicza ile linków nie zostało znalezionych w ogóle,
# tzn. po wysłaniu zapytania nie dostaliśmy żądnej odpwiedzi od sparql (wiersze oznaczone przez NOT FOUND URI)
not_found_uris = 0

for index, row in target.iterrows():
    try:
        counter = counter + 1
        if counter % 100 == 0:
            break

        print('aaa')

        table_id = row['TableID']
        column_id = row['ColumnID']
        row_id = row['RowID']
        
        # wczytanie odpowiedniej tabeli
        df = pd.read_csv(f"/home/ngiersz/Projects/zti/data/CEA_Round1/{table_id}.csv",
                    sep=',')
        
        # wyciągniecie wartości komórki, która nas interesuje
        cell_content = df.iloc[row_id-1:row_id, column_id:(column_id+1)].values[0][0]
        cell_content_copy = cell_content

        # usunięcie białych znaków z początku i końca stringa
        cell_content = cell_content.strip()
        

        # aktualnie nie używane, ale pewnie będzie
        # pos_tag() - kategoryzuje słowa, np. znajduje czasowniki, rzeczowniki, nazwy własne (jak imiona i nazwiska)
        # chodzi o to, że czasami odpowiedzi są w angielskiej dbpedii,
        # np. Alex Rodriguez wykrywa jako język ca - pewnie kataloński, 
        # ale nie ma jego odpowiednika w katalońskiej dbpedii, a w angielskiej jest
        # dlatego chcę wykrywać nazwy wkłasne jak imiona i nazwiska i wtedy zawsze szukać w angielksiej dbpedii,
        # ale z tym też był problem czasami i na razie nie myślałem nad tym dalej
        # NNP - to kategoria nazwy własnej, np. imię
        tags = pos_tag(cell_content.split())
        is_one_NNP = False
        for tag in tags:
            if tag[1] == "NNP":
                is_one_NNP = True
                break
        try:
            if is_one_NNP:
                lang = '@en'
            else:
                lang = str('@') + str(detect(cell_content))
        except Exception:
            lang = ''

    
        # pierwsze zapytanie - bierze zawartość sformatowanej komórki i wyszukuje linku,
        # na razie jest ustawiona na stałe angielska dbpedia (#en) z powodu tego co wcześniej opisałem
        query = 'SELECT ?resource WHERE { { ?resource rdfs:label ' + '"' + str(cell_content) + '"' + "@en" + '}'
        
        
        cell_contents_list = []
        cell_contents_list.append(cell_content)
        cell_contents_list.append(cell_content.capitalize())
        # pojawiały się wartości typy "Guardian The", a powinno być "The Gurdian"
        if re.search(" The$", cell_content):
            index = cell_content.find(" The")
            cell_contents_list.append("The " + cell_content[:index].replace(',','').strip())
            cell_contents_list.append(cell_content[:index].strip())
       
#         print(cell_contents_list)
        # zamienia wszystkie znaki odpowiadające kluczowi np. '[?]' na to co w wartości ''
        # każdy znak zamienia osobno. Nie wyszukuje stringa '[?]', tylko każdy znak po kolei szukam i zamieniam
        strings_to_replace_dict = [{'[?]':''}, {',':''}, {'-':' '}, {'?':''}] # dict = {chars_to_replace:new_char}
        for dicts in strings_to_replace_dict:
            for old_chars, new_char in dicts.items():
                cell_content_temp = cell_content
                for old_char in old_chars:
                    cell_content_temp = cell_content_temp.replace(old_char, new_char)
                if cell_content_temp not in cell_contents_list:
                    cell_contents_list.append(cell_content_temp)
        
#         for cell_content in cell_contents_list:
#             query = query + ' UNION { ?resource rdfs:label ' + '"' + str(cell_content) + '"' + "@en" + '} '
        
        lang = 'en'
        try:
            lang = detect(cell_content)
        except:
            pass
        
#         print(lang)
        if lang != 'en':
            for cell_content  in cell_contents_list:
#                 print(f'{cell_content} translated to {str(translator.translate(cell_content).text.title())}')
                query = 'SELECT ?resource WHERE { { ?resource rdfs:label ' + '"' + str(translator.translate(cell_content).text.title()) + '"' + "@en" + '}'
#                 query = query + ' UNION { ?resource rdfs:label ' + '"' + str(cell_content) + '"' + '@' + str(lang) + '} '
        query = query + '}'
            
            
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        
        # jeśli wystąpi błąd z połączeniem się z serwerem to próbujemy aż do skutku
        results = None
        while results is None:
            try:
                results = sparql.query().convert()
            except Exception as e:
#                 print("EXCEPTION:" + str(e))
#                 print("TAKI: " + str(cell_content))
                continue
        

        # jeśli nic nie znaleziono no to trudno, nie mamy linku, wstawiamy NOT FOUND URI
        if str(results['results']['bindings']).find("http://dbpedia.org/resource/") == -1:
            answers_dict['TableID'].append(table_id)
            answers_dict['ColumnID'].append(column_id)
            answers_dict['RowID'].append(row_id)
            answers_dict['URI'].append("NOT FOUND URI")
            # printy przydatne do sprawdzania jakich zawartości nam nie wyszukał,
            # trzeba analizowac i dodawać kolejne warunki, zmieniać cell_content na różne sposoby
#             print(table_id, column_id, row_id)
            print(cell_content_copy)
#             print(query)
            not_found_uris = not_found_uris + 1
            continue

        # jeśli jednak mamy jakies linki to wyciągamy go z jsona,
        # linków może być więcej niż 1, aktualnie bierzemy pierwszy lepszy
        results_list = results['results']['bindings']
        for result_dict in results_list:
    #         print(result_dict)
            uri = result_dict['resource']['value']
            match = re.match("http://dbpedia.org/resource/*", uri)
            
            # czasami pojawiają dziwne linki np. http://dbpedia.org/resource/Category:Max_Weber
            # my takich nie chcemy i musimy ich unikać
            # interesują nas tylko np.  http://dbpedia.org/resource/Max_Weber
            # więc to jest warunek pozbywajacy się wszystkich linków z Category:
            if match and uri[5:].find("Category:") == -1:
                answers_dict['TableID'].append(table_id)
                answers_dict['ColumnID'].append(column_id)
                answers_dict['RowID'].append(row_id)
#                 answers_dict['URI'].append(str(uri)
#                 formatted_uri = 'http://dbpedia.org/resource/' + str(urllib.parse.quote(uri[28:]))
#                 formatted_uri = formatted_uri.replace('-','%E2%80%93')
                answers_dict['URI'].append(uri)
                break
    except Exception as e:
        # na początku były problemy z danymi, naprawiłem
        # potem zaczęły pojawiać sę problemy, żę serwer czasami nie odpowiadał,
        # więc jak jest error to olewamy i przechodzimy do następnego wiersza
        # będzie trzeba zrobić, że jak serwer nie odpowie z jakiegoś powodu to trzeba ponowić to samo zapytanie
        # zamiast pomijać, bo tracimy wiersze
        exc_type, exc_obj, exc_tb = sys.exc_info()
#         print(table_id, column_id, row_id) # jaki plik, kolumna i wiersz
#         print(cell_content_copy) # zawartość komórki
#         print(e) # jaki błąd, teraz chyba tylko z brakiem połączenia z serwerem
#         print(exc_tb.tb_lineno) # w jakiej liniii błąd
        continue



# print(answers_dict)
result_df = pd.DataFrame(answers_dict)
end = time.time()
print(end-start)
print(not_found_uris)

aaa
aaa
aaa
aaa
aaa
aaa
School of Rock , The
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
Peter of Sebaste[?]
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
TREE SWALLOW
aaa
aaa
aaa
aaa
Fellbarrow
aaa
aaa
aaa
aaa
aaa
aaa
aaa
Life as A House
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
Snowy-headed robin-chat
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
Legend of Zelda: Majora's MaskThe Legend of Zelda: Majora's Mask
aaa
aaa
aaa
aaa
St Helena Pound
aaa
aaa
aaa
Godfather, The
aaa
aaa
Acacia longifolia
aaa
aaa
SingStar Pop Hits
aaa
aaa
46.87773299217224
11


In [23]:
print(result_df.shape)
print(target.shape)
result_df[:10]

(0, 4)
(8417, 3)


,TableID,ColumnID,RowID,URI


In [24]:
# zapisanie submission
# przez evaluator codes można sprawdzić wyniki
result_df.to_csv("/home/ngiersz/Projects/zti/data/my_submission3.csv",
                 index=False,
                 sep=',',
                header=False)

### NIEISTOTNE, TESTY ITP.

In [ ]:
df = pd.read_csv("D:\\Dokumenty\\Zastosowanie Technologii Informatycznych\\data\\CEA_Round1\\13719111_1_5719401842463579519.csv",
                sep=',')
df[:3]

In [ ]:
detect(df.iloc[0:1, 1:2].values[0][0])

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?label
    WHERE { <http://dbpedia.org/resource/Norway> rdfs:label ?label }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)
for result in results["results"]["bindings"]:
    print(result["label"]["value"])

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX ont: <http://dbpedia.org/ontology/>
    SELECT ?a, ?c
    WHERE { 
            ?a ont:location <http://dbpedia.org/resource/Delft> . 
            FILTER regex(?a, "Ikea", "i")
          }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)
# print(result['a'])
# print(result['b'])
# print(result['c'])

# for result in results["results"]["bindings"]:
#     print(type(result))

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sss = "dbr:Game_of_Thrones"
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery('''
ASK {e}
    VALUES (?r) {e} ({source}) {w}
        {e} ?r ?p ?o {w}
        UNION
        {e} ?s ?r ?o {w}
        UNION
        {e} ?s ?p ?r {w}
    {w} 
'''.format(source=sss, e="{", w="}"))
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sss = "dbr:Game_of_Thrones"
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery('''
ASK {
    VALUES (?r) { (dbr:Gra_o_tron) }
        { ?r ?p ?o }
        UNION
        { ?s ?r ?o }
        UNION
        { ?s ?p ?r }
    } 
''')
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)



In [ ]:
SELECT ?r ?p ?o
WHERE {
 VALUES (?r) { (dbr:Cat) }
        { ?r ?p ?o }
        UNION
        { ?s ?r ?o }
        UNION
        { ?s ?p ?r }
}

select ?s
where {
        { ?s rdfs:label "Kot domowy"@pl }
        UNION
        { "Kot domowy"@pl rdfs:label ?s }
} 

SELECT ?r
WHERE {
        { ?r rdfs:label "Kaspisches Meer"@de }

}

In [ ]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    SELECT ?resource
    WHERE { ?resource rdfs:label "Kaspisches Meer"@de }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
# print(results['results']['bindings'][1]['r']['value'])
# results = str(results['results']['bindings'])
print(str(results['results']['bindings']))
if str(results['results']['bindings']).find("http://dbpedia.org/resource/") == -1:
    print("nie ma")
else:
    print('jest')
# for result_dict in results_list:
#     uri = result_dict['resource']['value']
#     match = re.match("http://dbpedia.org/resource/*", uri)
#     if match:
#         print(uri)
#     else:
#         print("NIE")

# print(results['results']['bindings'])

In [ ]:
import re
match = re.match("http://dbpedia.org/resource/*",'http://dbpedia.org/resource/Caspian_Sea', re.M|re.I)
if match:
    print(match.group())